In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('kickstarter_desc.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'kickstarter_desc.pkl'

In [15]:
# step one, rename parsed
df.rename(columns = {"description":"parsed"}, inplace = True)

In [16]:
# step two, pull out the four columns
df[['description', 'img_count', 'vid_count', 'rewards']] = pd.DataFrame(df['parsed'].tolist(), index=df.index) 

In [17]:
## df[df['description'].str.contains('This Error:')].info()
## Uhh there's 184 errors and I may just drop them all. Pulling the URLs again doesn't seem to be worth it. There's
## multiple error that can result from this and 184/200,000 records isn't so bad. 


## Step three, drop parsed column
df.drop('parsed',axis = 1, inplace = True)

In [18]:
## Step four, drop rows that don't have all information
df.dropna(subset=['img_count', 'vid_count', 'rewards'], inplace = True)

In [19]:
## Step five, transforms "N/A" strings to NaNs, clean up empty vals
df.replace('N/A',np.NaN, inplace = True)

In [20]:
##step six, clean up empty / null values
df.description.replace('', None, inplace = True)

## It looks like for some descriptions, it's a long string of null characters that we may be able to ignore
## during analysis

In [21]:
## Step seven, clean up categories

# df.loc[10]['category']
# df.groupby(["category_name", "category_slug"]).size()
def cat_core(str):
    if "/" in str:
        split_list = str.split('/')
        core = split_list[0]
        return core
    else:
        return str
    
df['category_core'] = df['category_slug'].apply(cat_core)

In [22]:
## Step eight, clean up dates 

## df.loc[0]['created_at_date']

df['created_at_date'] = pd.to_datetime(df['created_at_date'])
df['deadline_date'] = pd.to_datetime(df['deadline_date'])
df['launched_at_date'] = pd.to_datetime(df['launched_at_date'])
df['state_changed_at_date'] = pd.to_datetime(df['state_changed_at_date'])

In [23]:
## Step Nine, USD Goal

df['usd_goal'] = df['goal'] * df['static_usd_rate']

In [24]:
## Step ten, percent reached

df['percent_goal'] = (df['pledged'] / df['goal']) * 100.00

In [25]:
## Step eleven, video_usage. 
## Determined that video usage is sparse and woudl rather see if they did use it or not in addition to how many

df['vid_usage'] = df['vid_count'] >= 1.0

In [26]:
## Step twelve, boolean whether they reached goal or not.

df['reach_goal'] = df['percent_goal'] >= 100.00

In [27]:
## Step thirteen, drop duplicates
df.drop_duplicates(subset = ['id', 'name'], inplace = True)

In [28]:
## Step fourteen, rewards translated
def convert_tool(values, rate):
    new_rewards = []
    for v in values:
        new_rewards.append(round(v * rate, 2))
    return new_rewards

df['usd_rewards'] = df.apply(lambda x: convert_tool(x['rewards'], x['static_usd_rate']), axis = 1)

In [29]:
## Step fifteen, did they reach the goal? boolean instead of quantitative
df['reach_goal'] = df['percent_goal'] >= 100.00

In [30]:
## Step sixteen, get the length of a description count & blurg

df['description_len'] = df['description'].str.len()
df['blurb_len'] = df['blurb'].str.len()
df['slug_len'] = df['slug'].str.len()

In [31]:
## Step eighteen, handling "state" of a project.
## We want to drop the 'live' state since it may not accurately represent a projects current status
## it's not fair if we took only half the 'live' projects that passed and not the ones who did not.
## We're also dropping the canceled and successful states. A project could be successful but canceled so it's
## not really telling. 
df = df[(df['state']=='successful') | (df['state'] == 'failed')]
df.drop(df[(df['state'] == 'failed') & (df['reach_goal'] == True)].index, inplace = True)
df.drop(df[(df['state'] == 'successful') & (df['reach_goal'] == False)].index, inplace = True)

In [32]:
df.to_pickle('kickstarter_clean.pkl')

## Analysis Cleaning Section

In [130]:
import pandas as pd
import numpy as np
df = pd.read_pickle('kickstarter_clean.pkl')

In [131]:
date_fields = ['created_at', 'deadline', 'launched_at', 'state_changed_at']
for fields in date_fields:
    df[fields] = pd.to_datetime(df[fields], unit = 's')

In [132]:
drop_list = ['backers_count', 'category', 'converted_pledged_amount',
            'created_at_date', 'created_at_time', 'creator', 'currency_trailing_code',
            'current_currency', 'deadline_date', 'deadline_time', 'fx_rate', 'goal', 'id', 
            'launched_at_date', 'launched_at_time', 'location',
            'name', 'pledged', 'profile', 'state_changed_at_date', 'state_changed_at_time', 'urls', 
            'usd_type', 'profile_id', 'location_localized_name', 
            'profile_project_id', 'profile_state', 'urls_web', 'rewards', 'category_slug',
            'state_changed_at', 'static_usd_rate', 'web_url']
## Values not useful for analysis

In [133]:
df_analysis = df.drop(drop_list, axis = 1)

In [134]:
df_analysis = df_analysis.reset_index(drop = True)

In [135]:
df_analysis['med_rewards'] = df_analysis.usd_rewards.apply(np.median)

/Users/Matt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [136]:
df_analysis['reward_len'] = df_analysis.usd_rewards.str.len()

In [137]:
df_analysis.spotlight = df_analysis.spotlight.astype(int)
df_analysis.staff_pick = df_analysis.staff_pick.astype(int)
df_analysis.reach_goal = df_analysis.reach_goal.astype(int)

In [138]:
df_analysis['failed'] = df_analysis.reach_goal == 0
df_analysis['failed'] = df_analysis['failed'].astype(int)

In [139]:
df_analysis['created_month'] = df_analysis.created_at.dt.strftime('%b')
df_analysis['created_weekday'] = df_analysis.created_at.dt.strftime('%a')

In [140]:
df_analysis['deadline_month'] = df_analysis.deadline.dt.strftime('%b')
df_analysis['deadline_weekday'] = df_analysis.deadline.dt.strftime('%a')

In [141]:
df_analysis['length_of_project'] = (df_analysis.deadline - df_analysis.created_at).dt.days

In [143]:
df_analysis.to_pickle('kickstarter_analysis.pkl')